In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from huggingface_hub import notebook_login
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer

MODEL_NAME = "meta-llama/Llama-2-7b-hf"

model = LlamaForCausalLM.from_pretrained(
    MODEL_NAME,
    return_dict=True,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

model = model.eval()
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)  
tokenizer = LlamaTokenizer.from_pretrained(MODEL_NAME)


# Define PAD Token = BOS Token
tokenizer.pad_token = tokenizer.bos_token
model.config.pad_token_id = model.config.bos_token_id



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /home/abdelrahman.sadallah/local/cuda-11.7/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.9
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/abdelrahman.sadallah/.conda/envs/nlp/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/abdelrahman.sadallah/.conda/envs/nlp/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: /home/abdelrahman.sadallah/.conda/envs/nlp did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


In [14]:





prompt = """### Instruction: I will give you a cryptic crossword which is a crossword puzzle in which each clue is a word puzzle. Your task is to solve this clue, and to give five possible answers for it. The number of charachters in the answer should be same as the numbers in the parenthesis in the clue. output 5 generations. The following are few examples to help you.\n\n### Input:\nDesk register taken no further than Ozzie? (7)\n\n### Output:\nrolltop\n\n### Input:\nWhat's missing in a fight is obscure (5,3)"""
predictions = []
labels = []

torch.cuda.empty_cache()



encoding = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **encoding,
        max_new_tokens=256,
        temperature=0.00001,
        generation_config=  generation_config,
    )  

answer_tokens = outputs[:, encoding.input_ids.shape[1] :]

output_text = tokenizer.batch_decode(answer_tokens, skip_special_tokens=True)





In [15]:
for i in output_text:

    lines = i.split('\n')
    for i,l in enumerate(lines):
        if l=='### Output:':
            print(lines[i+1])
            
# print(output_text[0])

absent
shocked
shocked
shocked
shocked
shocked
shocked
shocked
shocked


In [8]:
answer_tokens

tensor([[   13,    13,  2277, 29937, 10604, 29901,    13, 29874,   282,  3322,
            13,    13,  2277, 29937, 10567, 29901,    13, 29909,   767, 29915,
         29879,   263,   767,   363,   263, 29915,   393,   313, 29945, 29897,
            13,    13,  2277, 29937, 10604, 29901,    13, 29874,   767,    13,
            13,  2277, 29937, 10567, 29901,    13, 29909,   767, 29915, 29879,
           263,   767,   363,   263, 29915,   393,   313, 29945, 29897,    13,
            13,  2277, 29937, 10604]], device='cuda:0')